# Equation d'advection

Nous considérons le problème de l'advection en dimension 1 d'espace posé sur un domaine périodique

\begin{equation}
\left\lbrace
\begin{aligned}
&\partial_t u(t, x) + c \partial_x u(t,x) = 0, && x\in[0,1], \quad t>0,\\
&u(0,x) = u_0(x), && x\in[0,1],\\
&u(t,0) = u(t,1), && t\geqslant 0,
\end{aligned}
\right.
\end{equation}

où $c\in\mathbb{R}$ est la vitesse supposée connue.

Testez le programme proposé dans ce notebook puis essayez d'ajouter de nouveaux schémas afin de visualiser leur comportement dans le cas de cette équation.

In [1]:
%%file TP01_advection.hpp

/*****************************************************************************************
*
* classes to define the numerical scheme of the simulation
*
*****************************************************************************************/
class scheme
{
private:
    double _dt;          // time step fixed by the CFL condition
public:
    std::string method = "";
    int bl = -1;         // boundary layer size
    double CFL;          // CFL = max(c) * dt/dx (Courant-Friedrichs-Levy parameter)
    double c;            // velocity
    double dt, dx;       // time step and spatial step
    std::size_t Nx;      // number of points
    std::vector<double> u_glob;    
    
    scheme() = default; // constructeur par defaut
    scheme(const scheme&) = default; // constructeur par defaut
    scheme(parameters);
    
    void init(solution1D&);
    void u2uglob(solution1D&);
    void bc();
    void onetimestep(solution1D&);
    virtual void formula(solution1D&){std::cout << "virtual formula" << std::endl;}
    void run(solution1D&, double);
    
};

// constructor
scheme::scheme(parameters param)
{
    // CFL
    auto it_CFL = param.find("CFL");
    if (it_CFL != param.end())
        CFL = it_CFL->second;
    else
    {
        std::cout << "CFL unknown\n";
        CFL = 1.;
    }
    // velocity c
    auto it_c = param.find("velocity");
    if (it_c != param.end())
        c = it_c->second;
    else
    {
        std::cout << "velocity unknown\n";
        c = 1.;
    }
}

void scheme::init(solution1D& S)
{
    if (bl>=0) // copy the vector solution u into a global vector u_glob with phantom cells
    {
        u_glob.resize(S.m.Nx + 2*bl);
        u2uglob(S);
    }
    else if (bl<0)
        std::cout << "Error in scheme init" << std::endl;
    Nx = S.m.Nx;
    dx = S.m.dx;
    _dt = CFL/std::fabs(c) * dx;
    dt = _dt;
}

void scheme::u2uglob(solution1D& S)
{
    std::copy(S.u.begin(), S.u.end(), std::next(u_glob.begin(), bl));
}

void scheme::bc()
{
    auto it_begin = std::next(u_glob.begin(), bl);
    auto it_end = std::prev(u_glob.end(), bl);
    // right boundary condition: periodic
    std::copy(std::next(u_glob.begin(), bl), 
              std::next(u_glob.begin(), 2*bl),
              std::prev(u_glob.end(), bl));
    // left boundary condition: periodic
    std::copy(std::prev(u_glob.end(), 2*bl),
              std::prev(u_glob.end(), bl),
              u_glob.begin());
}

void scheme::onetimestep(solution1D &S)
{
    u2uglob(S);    // copy u into u_glob
    bc();          // apply the boundary conditions
    formula(S);    // apply the scheme formula
    S.t += dt;  // modify the time
}

void scheme::run(solution1D& S, double tf)
{
    while (S.t < tf)
    {
        dt = std::min(tf-S.t, _dt);
        onetimestep(S);
    }
}

std::ostream& operator<<(std::ostream& out, scheme& s)
{
    out << "------------------------------------------------" << std::endl;
    out << "scheme informations " << std::endl;
    out << "------------------------------------------------" << std::endl;
    out << "    CFL:               " << s.CFL << std::endl;
    out << "    velocity:          " << s.c << std::endl;
    out << "    method:            " << s.method << std::endl;
    out << "------------------------------------------------" << std::endl;
    return out;
}

class LF : public scheme
{
public:
    LF(parameters param) : scheme(param)
    {
        method = "Lax-Friedrichs";
        bl = 1;
    }
    void formula(solution1D& S)
    {
        auto alpha = .5*c*dt/dx;
        for (auto i=0, ig=bl; i<Nx; ++i, ++ig)
            S.u[i] = .5*(u_glob[ig+1] + u_glob[ig-1]) - alpha * (u_glob[ig+1] - u_glob[ig-1]);
    }
};

// BEGIN SOLUTION
class UW : public scheme
{
public:
    UW(parameters param) : scheme(param)
    {
        method = "Upwind";
        bl = 1;
    }
    void formula(solution1D& S)
    {
        auto alpha = c*dt/dx;
        if (c>0)
        {
            for (auto i=0, ig=bl; i<Nx; ++i, ++ig)
                S.u[i] = u_glob[ig] - alpha * (u_glob[ig] - u_glob[ig-1]);
        }
        if (c<0)
        {
            for (auto i=0, ig=bl; i<Nx; ++i, ++ig)
                S.u[i] = u_glob[ig] - alpha * (u_glob[ig+1] - u_glob[ig]);
        }

    }
};

class LW : public scheme
{
public:
    LW(parameters param) : scheme(param)
    {
        method = "Lax-Wendroff";
        bl = 1;
    }
    void formula(solution1D& S)
    {
        auto alpha = .5*c*dt/dx, beta = 2*alpha*alpha;
        for (auto i=0, ig=bl; i<Nx; ++i, ++ig)
            S.u[i] = u_glob[ig] 
                - alpha * (u_glob[ig+1] - u_glob[ig-1]) 
                + beta * (u_glob[ig+1] - 2*u_glob[ig] + u_glob[ig-1]);
    }
};
// END SOLUTION

Overwriting TP01_advection.hpp


In [2]:
#include <iostream>
#include <vector>

#include <chrono>
#include <thread>

#include "xplot/xfigure.hpp"
#include "xplot/xmarks.hpp"
#include "xplot/xaxes.hpp"

using parameters = std::map<std::string, double>;

#include "mesh1D.hpp"
#include "solution1D.hpp"
#include "TP01_advection.hpp"

In [3]:
// parameters declaration
parameters p = {
    {"CFL", .75}, 
    {"velocity", 1}, 
    {"xmin", -1},
    {"xmax", 1.},
    {"Nx", 1024},
    {"cell-centered", 1}
};

In [4]:
double u0(double x)
{
    double xmin = p["xmin"], xmax = p["xmax"];
    auto l = 0.75 * xmin + 0.25 * xmax;
    auto r = 0.5 * xmin + 0.5 * xmax;
    auto xm = x - std::floor((x-xmin)/(xmax-xmin))*(xmax-xmin); // periodic function
    if ((xm>l) && (xm<r))
        return 1.;
    else
        return 0.;
}

In [5]:
auto u(double t, std::vector<double> x)
{
    std::vector<double> y(x.size());
    double c = p["velocity"];
    for (auto k = 0; k < x.size(); ++k)
        y[k] = u0(x[k] - c*t);
    return y;
}

In [6]:
LF s(p);        // Lax-Friedrichs scheme
solution1D S(p);  // numerical solution
S.init(u0);     // initialize the numerical solution
s.init(S);      // initialize the scheme
double tf;      // final time
std::size_t Nt; // number of plots

In [7]:
xpl::linear_scale sx, sy;
sx.min = p["xmin"], sx.max = p["xmax"];
sy.min = S.min(), sy.max = S.max();

auto ax_x = xpl::axis_generator(sx)
    .label("x")
    .finalize();
auto ax_y = xpl::axis_generator(sy)
    .orientation("vertical")
    .side("left")
    .finalize();
auto line_e = xpl::lines_generator(sx, sy)
    .x(S.m.x)
    .y(S.u)
    .colors(std::vector<std::string>({"orange"}))
    .labels(std::vector<std::string>({"exact"}))
    .display_legend(true)
    .finalize();
auto line_a = xpl::lines_generator(sx, sy)
    .x(S.m.x)
    .y(S.u)
    .colors(std::vector<std::string>({"navy"}))
    .labels(std::vector<std::string>({s.method}))
    .display_legend(true)
    .finalize();
auto fig = xpl::figure_generator()
    .padding_x(0.025)
    .padding_y(0.1)
    .title("solution at t = " + std::to_string(0))
    .legend_location("top-left")
    .finalize();

fig.add_mark(line_e);
fig.add_mark(line_a);
fig.add_axis(ax_x);
fig.add_axis(ax_y);
fig.display();

A Jupyter widget

In [8]:
S.init(u0);
tf = 2.;
Nt = 100;
for (auto k = 1; k <= Nt ; ++k)
{
    double t = k*tf/Nt;
    s.run(S, t);
    line_a.y = S.u;
    line_e.y = u(S.t, S.m.x);
    fig.title = "Advection at t = " + std::to_string(S.t);
    std::this_thread::sleep_for(std::chrono::milliseconds(10));
}